In [1]:
import json
import numpy as np
import wowsims

# Load Fury Raid Sim Request JSON

In [2]:
f = open('data/fury-human-bis-p3.json')
settings = json.load(f)

# Set environment and settings

In [3]:
from fury import *

def reset():
    wowsims.new(json.dumps(settings).encode('utf-8'))


# Iterations are currently capped at 3000
iterations = settings['simOptions']['iterations']

duration = settings['encounter']['duration']

spell_id_lists = {
        '0' : "Melee",
        '23881' : "Bloodthirst",
        '1680' : "Whirlwind",
        '47475' : "Slam",
        '47450' : "Heroic Strike",
        '47471' : "Execute",
        '12867': "Deep Wounds",
        '12292' : "Death Wish",
        '1719' : "Recklessness",
        '64382' : "Shattering Throw",
        '54758' : "Engi Gloves",
        '2457' : "Battle Stance",
        '2458' : "Berserker Stance",
        '2687' : 'Bloodrage',
        '2825' : 'Bloodlust',
        '47465': 'Rend',
        '7384': 'Overpower',
        '44949': 'Whirlwind OH',
}

# Default sim agent
Default sim agent is the hardcoded agent from the sim

In [6]:
settings['simOptions']['interactive'] = False
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 11782.084625708363


In [7]:
settings['simOptions']['interactive'] = False

while not wowsims.step():
    pass
json_string = wowsims.getSpellMetrics()
cast_metrics = json.loads(json_string)
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Melee: [1, 1, 1, 1, 6, 63]
Death Wish: [1]
Deep Wounds: [115]
Whirlwind: [13]
Recklessness: [1]
Bloodthirst: [27]
Battle Stance: [10]
Berserker Stance: [10]
Bloodrage: [3]
Bloodlust: [1]
Whirlwind OH: [13]
Heroic Strike: [57]
Rend: [4]
Execute: [4]
Slam: [15]
Engi Gloves: [2]
Shattering Throw: [1]
Overpower: [6]


# PPO Agent

## Load Trained PPO Model

In [4]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.registry import register_env
from fury_sim_env import FurySimEnv
from gymnasium import make
from gymnasium.envs.registration import register

def env_creator(env_config):
    return FurySimEnv(...)

register_env("FurySimEnv", env_creator)

config = PPOConfig()\
            .rollouts(num_rollout_workers=1)
algorithm = config.build(env="FurySimEnv")
algorithm.restore("models\Fury\PPO\PPO_FurySimEnv_9899e_00000_0_2023-05-01_00-08-15\checkpoint_000008")

register(id="FurySimEnv", entry_point="fury_sim_env:FurySimEnv")
env = make("FurySimEnv")

2023-05-01 00:28:02,407	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(RolloutWorker pid=11448) 2023-05-01 00:28:07,536	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
2023-05-01 00:28:07,579	WARNING util.py:67 -- Install gputil for GPU system monitoring.
2023-05-01 00:28:07,582	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory models\Fury\PPO\PPO_FurySimEnv_9899e_00000_0_2023-05-01_00-08-15\checkpoint_000008\.! Trying to extract checkpoint info from other files found in that dir.
2023-05-01 00:28:07,669	WARNING algorithm_config.py:635 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2023-05-01 00:28:07,689	INFO trainable.py:913 -- Restored on 127.0.0.1 from checkpoint: models\Fury\PPO\PPO_FurySimEnv_9899e_00000_0_2023-05-01_00-08-15\checkpoint_00

## Run PPO Model

In [5]:
for i in range(100):
    observation, info = env.reset()
    terminated = False
    reward = 0
    batch = []
    dps_results = np.array([])
    metrics_batch = []
    debug_logs = []
    debug_logs_batch = []
    while not terminated:
        action = algorithm.compute_single_action(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        batch.append(np.concatenate((observation, [action], [reward])))
        debug_logs.append(info['debug log'])
    debug_logs_batch.append(debug_logs)
    dps_results = np.append(dps_results, info['dps'])
    metrics_batch.append(info['spell metrics'])

In [6]:
median = np.argsort(dps_results)[len(dps_results)//2]
print(f'Average DPS: {dps_results[median]}')

cast_metrics = json.loads(metrics_batch[median])
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Average DPS: 9321.704502234199
Melee: [1, 52, 51]
Death Wish: [1]
Deep Wounds: [98]
Whirlwind: [9]
Recklessness: [1]
Bloodthirst: [14]
Bloodrage: [3]
Bloodlust: [1]
Whirlwind OH: [9]
Heroic Strike: [33]
Execute: [2]
Slam: [24]
Engi Gloves: [2]
Shattering Throw: [1]


### Debug Log

In [7]:
actions_id = {
    0: "Bloodthirst", 
    1: "Whirlwind",
    2: "Slam",
    3: "HeroicStrike",
    4: "Execute",
    5: "DeathWish",
    6: "Recklessness",
    7: "ShatteringThrow",
    8: "Bloodrage",
    9: "EngiGlove",
    10: "Bloodlust",
}

In [11]:
import pandas as pd

df = pd.DataFrame(debug_logs_batch[median], columns=['Timestamp', 'Action', 'Successful Cast', 'Damage Done', 'Total Damage Done', 'Rage'])
df['Action'] = df['Action'].map(actions_id)
df['DPS'] = df['Total Damage Done'].div(df['Timestamp'])
df

,Timestamp,Action,Successful Cast,Damage Done,Total Damage Done,Rage,DPS
0,0.010000,HeroicStrike,True,11457.238327,1.145724e+04,54.805632,1.145724e+06
1,0.020000,Slam,True,0.000000,1.145724e+04,54.805632,5.728619e+05
2,1.530000,Bloodthirst,True,19293.494877,3.075073e+04,51.074678,2.009852e+04
3,3.030000,Whirlwind,True,21570.054997,5.232079e+04,75.000000,1.726759e+04
4,4.695287,Whirlwind,False,0.000000,6.464548e+04,100.000000,1.376816e+04
...,...,...,...,...,...,...,...
240,119.047451,Whirlwind,True,7979.560846,1.129752e+06,75.000000,9.489931e+03
241,121.047451,DeathWish,False,0.000000,1.133531e+06,100.000000,9.364356e+03
242,121.231073,Bloodthirst,True,13196.984524,1.146728e+06,80.000000,9.459030e+03
243,122.731073,Slam,True,10184.123378,1.156913e+06,85.000000,9.426403e+03


In [12]:
df.to_csv("debug_logs/debug_log_1.csv")

# Spell Debug

In [15]:
Spells.register()
Auras.register()
TargetAuras.register()

settings['simOptions']['interactive'] = True
damages = np.array([])
spell_metrics = []

for i in range(3000):
    reset()
    while not wowsims.step():
        if wowsims.needsInput():
            wowsims.trySpell(Spells.Slam)
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)
    spell_metrics.append(wowsims.getSpellMetrics())

median = np.argsort(damages)[len(damages)//2]
print(f'Average DPS: {damages[median] / duration}')

Average DPS: 4925.963917374689


In [16]:

json_string = spell_metrics[median]
cast_metrics = json.loads(json_string)
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Melee: {'Casts': 1, 'Misses': 0, 'Hits': 0, 'Crits': 0, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 0, 'TotalThreat': 0, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 0}
Deep Wounds: {'Casts': 51, 'Misses': 0, 'Hits': 51, 'Crits': 0, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 106393.62398275181, 'TotalThreat': 74475.53678792623, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 0}
Slam: {'Casts': 67, 'Misses': 0, 'Hits': 29, 'Crits': 38, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 409385.55050488695, 'TotalThreat': 293135.885353421, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 102000000000}
